In [1]:
import numpy as np
from alphastable import *
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'alphastable'

In [ ]:
sigma = 1
d = 2
N = 100
alpha = 1.3
beta = 1
gamma = np.cos((np.pi * alpha) / 4) ** (2/alpha)
delta = 0
G = np.random.normal(0, sigma, size=[N, d])
A = alphastable(N, 1, alpha/2, beta, gamma, delta, 1)
X = np.sqrt(A) * G

In [ ]:
n = 5
E = np.arange(0, 2*np.pi, np.pi / n)
R = np.arange(10, 60, 10)
Gamma = np.ones((len(E), len(R)))
Gamma[:] = numpy.nan
for k in range(len(E)):
    e = E[k]
    for j in range(len(R)):
        r = R[j]
        counter_1 = 0
        counter_2 = 0
        for i in range(len(X)):
            if np.arctan2(X[i][0], X[i][1]) < e and np.arctan2(X[i][0], X[i][1]) > 0 and np.linalg.norm(X[i]) > r:
                counter_1 += 1
            if np.linalg.norm(X[i]) > r:
                counter_2 += 1
        if counter_2 != 0:
            Gamma[k, j] = counter_1 / counter_2


In [28]:
Gamma

array([[0. , 0. , 0. , nan, nan],
       [0.2, 0.5, 0. , nan, nan],
       [0.4, 0.5, 0. , nan, nan],
       [0.4, 0.5, 0. , nan, nan],
       [0.4, 0.5, 0. , nan, nan],
       [0.4, 0.5, 0. , nan, nan],
       [0.4, 0.5, 0. , nan, nan],
       [0.4, 0.5, 0. , nan, nan],
       [0.4, 0.5, 0. , nan, nan],
       [0.4, 0.5, 0. , nan, nan]])